In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt

img_width = 1920
img_height = 1080

frame_range = [0, 99999]

content = []
camera_device = "0619-3"

label_folder = r"predict_results/" + camera_device
output_folder = r"filter_results/" + camera_device

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

txt_files = [f for f in os.listdir(label_folder) if f.lower().endswith('.txt')]

pattern = re.compile(r"^" + camera_device + r"_(\d+)\.txt$")

for txt_file in txt_files:
    match = pattern.match(txt_file)
    if not match:
        continue
    
    file_path = os.path.join(label_folder, txt_file)
    with open(file_path, 'r') as f:
        lines = f.readlines()
        if len(lines) == 0:
            os.remove(file_path)
        else:
            file_number = int(match.group(1))
            numbers = [float(num) for num in lines[0].strip().split()[1:]]
            content.append([file_number] + numbers + [0])

content = np.array(content)
content[:, 1] = content[:, 1] * img_width
content[:, 2] = content[:, 2] * img_height
content[:, 3] = content[:, 3] * img_width
content[:, 4] = content[:, 4] * img_height
content = content[content[:, 0].argsort()]
original_content = content.copy()

def distance(frame1, frame2):
    x1, y1, w1, h1 = frame1[1:5]
    x2, y2, w2, h2 = frame2[1:5]
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

for i in range(1, len(content)):
    t1 = content[i - 1]
    t2 = content[i]
    if distance(t1, t2) <= 10 and t2[0]-t1[0] == 1:
        t1[-1] = 1
        t2[-1] = 1
    elif distance(t1, t2) <= 20 and t2[0]-t1[0] == 2:
        t1[-1] = 1
        t2[-1] = 1
    elif distance(t1, t2) <= 30 and t2[0]-t1[0] == 3:
        t1[-1] = 1
        t2[-1] = 1

drone_distances = []
for i in range(1, len(content)):
    if content[i - 1, -1] == 1 and content[i, -1] == 1 and content[i, 0] - content[i - 1, 0] == 1:
        dist = distance(content[i - 1], content[i])
        drone_distances.append(dist)

drone_distances = np.array(drone_distances)

img_width_left = img_width * 0.01
img_width_right = img_width * 0.99
img_height_up = img_height * 0.01
img_height_down = img_height * 0.99
sorted_drone_distances = sorted(drone_distances)

Q1 = np.percentile(sorted_drone_distances, 25)
Q3 = np.percentile(sorted_drone_distances, 75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

non_outlier_distances = [d for d in sorted_drone_distances if lower_bound <= d <= upper_bound]

max_non_outlier_distance = 2.5 * max(non_outlier_distances) if non_outlier_distances else None

i = 0
while i < len(content) - 1:
    if content[i][-1] == 1:
        j = i + 1
        while j < len(content) and content[j][-1] != 1:
            j += 1
        if j < len(content):
            
            time_length = int(content[j][0] - content[i][0])
            t1 = content[i]
            t2 = content[j]
            x1_in_range = t1[1] > img_width_left and t1[1] < img_width_right
            y1_in_range = t1[2] > img_height_up and t1[2] < img_height_down
            x2_in_range = t2[1] > img_width_left and t2[1] < img_width_right
            y2_in_range = t2[2] > img_height_up and t2[2] < img_height_down

            if (time_length <= 180 and x1_in_range and y1_in_range and x2_in_range and y2_in_range) or time_length <= 60:
                if distance(t1, t2) <= max_non_outlier_distance * time_length:
                    for k in range(1, time_length):
                        new_frame = [
                            t1[0] + k,
                            t1[1] + (t2[1] - t1[1]) / time_length * k,
                            t1[2] + (t2[2] - t1[2]) / time_length * k,
                            t1[3] + (t2[3] - t1[3]) / time_length * k, 
                            t1[4] + (t2[4] - t1[4]) / time_length * k,
                            t1[5],
                            1
                        ]
                        content = np.insert(content, i + k, new_frame, axis=0)
                    i += time_length - 1
    i += 1

for frame in content:
    frame_number = int(frame[0])
    if frame_number < frame_range[0]:
        continue
    if frame_number >= frame_range[1]:
        break
    output_file = os.path.join(output_folder, f"{camera_device}_{frame_number:05d}.txt")
    if frame[-1] == 1:
        with open(output_file, 'w') as f:
            f.write(f"0 {frame[1]/img_width} {frame[2]/img_height} {frame[3]/img_width} {frame[4]/img_height}\n")

print("Data processing and saving completed.")

# Merge into a single file

In [ ]:
import os

def process_files(input_folder, output_file):
    files = [f for f in os.listdir(input_folder) if f.endswith(".txt")]
    
    files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    with open(output_file, 'w') as out_file:
        for file in files:
            frame = file.split('_')[-1].split('.')[0]
            with open(os.path.join(input_folder, file), 'r') as in_file:
                line = in_file.readline().strip()
                values = line.split()
                if len(values) >= 3:
                    x = float(values[1]) * 1920
                    y = float(values[2]) * 1080
                    out_file.write(f"{int(frame)}\t{x:.1f}\t{y:.1f}\n")

output_file = f'filter_results/{camera_device}.txt'
process_files(output_folder, output_file)
print('Merge completed!')